In [1]:
# Parameters
REGION_PARAMETRO = "Afghanistan"
NOMBRE_ARCHIVO_SALIDA = "output/indicadores_Afghanistan.csv"


In [2]:
# --- Celda 1: Parámetros e Inicialización (¡ETIQUETAR como 'parameters' en Jupyter!) ---
import pandas as pd
import numpy as np
import os 
from IPython.display import display

# Variables inyectadas por el Maestro
REGION_PARAMETRO = "Argentina"
NOMBRE_ARCHIVO_SALIDA = f"output/indicadores_{REGION_PARAMETRO.replace(' ', '_')}.csv"

# URL de los datos públicos
URL_COVID_DATA = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"

# Leer el CSV
try:
    df = pd.read_csv(URL_COVID_DATA)
    print(f"✅ Datos brutos cargados. Filas totales: {len(df)}")
except Exception as e:
    print(f"❌ Error al cargar los datos: {e}")

# Requisito: Imprimir información del archivo recién migrado
print("\n--- 💾 Información de Tipos de Datos y Memoria ---")
df.info()

print("\n--- 📝 Primeras filas del DataFrame (Province/State visible) ---")
display(df.head()) 

print("\n--- 🏷️ Tipos de Datos por Columna ---")
print(df.dtypes)
print("-------------------------------------------------------------------")

✅ Datos brutos cargados. Filas totales: 289

--- 💾 Información de Tipos de Datos y Memoria ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Columns: 1147 entries, Province/State to 3/9/23
dtypes: float64(2), int64(1143), object(2)
memory usage: 2.5+ MB

--- 📝 Primeras filas del DataFrame (Province/State visible) ---


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288



--- 🏷️ Tipos de Datos por Columna ---
Province/State     object
Country/Region     object
Lat               float64
Long              float64
1/22/20             int64
                   ...   
3/5/23              int64
3/6/23              int64
3/7/23              int64
3/8/23              int64
3/9/23              int64
Length: 1147, dtype: object
-------------------------------------------------------------------


In [3]:
# --- Celda 2: Detección y Limpieza de Calidad de Datos (EDA/ETL Inicial) ---

# Tarea 1: Normalización Selectiva de Nombres de Columnas
df.rename(columns={
    'Country/Region': 'country_region',
    'Lat': 'lat',
    'Long': 'long'
}, inplace=True)
print("✅ Nombres de columnas clave normalizados. 'Province/State' se mantuvo con su nombre original.")

# ----------------------------------------------------
# ANÁLISIS 2: Detección de Valores Nulos (NULL/NaN)
# ----------------------------------------------------
print("\n--- 🔎 ANÁLISIS DE NULOS ---")
df_nulos = df.isnull().sum()
df_nulos_resumen = pd.DataFrame({
    'Nulos_Conteo': df_nulos,
    'Nulos_Porcentaje': (df_nulos / len(df) * 100).round(2)
})

df_nulos_resumen = df_nulos_resumen[df_nulos_resumen['Nulos_Conteo'] > 0]
display(df_nulos_resumen)

# Tarea de Corrección: Imputación de Nulos en 'Province/State'
COLUMNA_PROVINCIA = 'Province/State' 
if COLUMNA_PROVINCIA in df_nulos_resumen.index:
    conteo_nulos_provincia = df_nulos_resumen.loc[COLUMNA_PROVINCIA, 'Nulos_Conteo']
    df[COLUMNA_PROVINCIA] = df[COLUMNA_PROVINCIA].fillna('')
    print(f"✅ Se corrigieron {conteo_nulos_provincia:,} nulos en '{COLUMNA_PROVINCIA}' imputando con cadena vacía.")
else:
    conteo_nulos_provincia = 0
    
# ----------------------------------------------------
# ANÁLISIS 3: Detección de Registros Duplicados
# ----------------------------------------------------
print("\n--- 🔎 ANÁLISIS DE DUPLICADOS ---")
conteo_duplicados_raw = df.duplicated().sum()

if conteo_duplicados_raw > 0:
    df.drop_duplicates(inplace=True)
    print(f"⚠️ Se eliminaron {conteo_duplicados_raw} registros duplicados EXACTOS.")
else:
    print("✅ No se encontraron registros duplicados exactos en el dataset.")
    
VARIABLES_LIMPIEZA = {
    'Nulos_Imputados_Provincia': conteo_nulos_provincia,
    'Registros_Duplicados_Eliminados': conteo_duplicados_raw
}

✅ Nombres de columnas clave normalizados. 'Province/State' se mantuvo con su nombre original.

--- 🔎 ANÁLISIS DE NULOS ---


,Nulos_Conteo,Nulos_Porcentaje
Province/State,198,68.51
lat,2,0.69
long,2,0.69


✅ Se corrigieron 198 nulos en 'Province/State' imputando con cadena vacía.

--- 🔎 ANÁLISIS DE DUPLICADOS ---


✅ No se encontraron registros duplicados exactos en el dataset.


In [4]:
# --- Celda 3: ETL, Limpieza de Series de Tiempo y Cálculo de Indicadores ---

# Tarea 1: Transformación de Ancho a Largo (Melt)
columnas_fecha = df.columns[df.columns.get_loc('long') + 1:]
df_melted = df.melt(
    id_vars=['Province/State', 'country_region', 'lat', 'long'], 
    value_vars=columnas_fecha,
    var_name='date',
    value_name='confirmed_cases'
)

# Tarea 2: Conversión de Tipos y Filtrado
# Conversión robusta de fechas
df_melted['date'] = pd.to_datetime(df_melted['date'], errors='coerce') 
df_melted.dropna(subset=['date'], inplace=True) # Elimina filas donde la fecha no se pudo convertir

# Filtrado por la región inyectada
df_region = df_melted[df_melted['country_region'] == REGION_PARAMETRO].copy()

# --- VERIFICACIÓN CRÍTICA ---
if df_region.empty:
    print(f"⚠️ No se encontraron datos para la región '{REGION_PARAMETRO}'. Saltando cálculos.")
    # Si no hay datos, inicializamos las variables de resultado en cero para que la Celda 4 no falle
    total_acumulado = 0
    max_diario = 0
    promedio_7_dias = 0.0
    conteo_errores_diarios = 0
else:
    # Tarea 3: Agregación Diaria y Casos Nuevos
    df_daily = df_region.groupby('date')['confirmed_cases'].sum().reset_index()
    # Cálculo de casos diarios nuevos
    df_daily['daily_new_cases'] = df_daily['confirmed_cases'].diff().fillna(0).astype(int)

    # Tarea 4: Limpieza de Casos Diarios Negativos (Correcciones de Fuente)
    conteo_errores_diarios = len(df_daily[df_daily['daily_new_cases'] < 0])
    df_daily.loc[df_daily['daily_new_cases'] < 0, 'daily_new_cases'] = 0
    print(f"✅ Se corrigieron {conteo_errores_diarios} días con caídas en casos acumulados.")

    # Tarea 5: Cálculo de Indicadores Finales
    total_acumulado = df_daily['confirmed_cases'].iloc[-1]
    max_diario = df_daily['daily_new_cases'].max()
    promedio_7_dias = df_daily['daily_new_cases'].tail(7).mean().round(2)
    
print(f"✅ Indicadores calculados para {REGION_PARAMETRO}.")

# --- Creación del DataFrame de Resumen (siempre se crea, con ceros si es necesario) ---
df_resumen = pd.DataFrame({
    'Region': [REGION_PARAMETRO],
    'Total_Acumulado': [total_acumulado],
    'Max_Diario': [max_diario],
    'Promedio_Ultimos_7_Dias': [promedio_7_dias],
    'Dias_Corregidos_Negativos': [conteo_errores_diarios],
    'Nulos_Imputados_Provincia': [VARIABLES_LIMPIEZA['Nulos_Imputados_Provincia']],
    'Duplicados_Eliminados': [VARIABLES_LIMPIEZA['Registros_Duplicados_Eliminados']]
})

✅ Se corrigieron 0 días con caídas en casos acumulados.
✅ Indicadores calculados para Argentina.


C:\Users\Goku\AppData\Local\Temp\ipykernel_10452\1070411051.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_melted['date'] = pd.to_datetime(df_melted['date'], errors='coerce')


In [5]:
# --- Celda 4: Generar Archivo de Resumen y Visualización ---

# 1. Verificación de seguridad: Asegura que df_resumen existe, no está vacío y es un DataFrame
if 'df_resumen' in locals() and isinstance(df_resumen, pd.DataFrame) and not df_resumen.empty:
    
    # 2. Guardar el DataFrame de resumen en un archivo CSV
    df_resumen.to_csv(NOMBRE_ARCHIVO_SALIDA, index=False)

    # 3. Visualizar el resultado para auditoría (Papermill)
    print(f"🎉 ¡Proceso de análisis completado para {REGION_PARAMETRO}!")
    print(f"✅ Archivo de indicadores generado en: {NOMBRE_ARCHIVO_SALIDA}")
    print("\n--- 📊 VISUALIZACIÓN DEL ARCHIVO CSV GENERADO (df_resumen) ---")
    display(df_resumen) 
    print("---------------------------------------------------------------")

else:
    # 4. Mensaje de error si la Celda 3 (cálculos) falló
    print(f"❌ ERROR CRÍTICO: El CSV NO fue creado para {REGION_PARAMETRO}.")
    print("La tabla de resumen (df_resumen) no existe, está vacía o es inválida.")
    print("Por favor, revise la Celda 3 para ver el error de cálculo.")

🎉 ¡Proceso de análisis completado para Argentina!
✅ Archivo de indicadores generado en: output/indicadores_Argentina.csv

--- 📊 VISUALIZACIÓN DEL ARCHIVO CSV GENERADO (df_resumen) ---


,Region,Total_Acumulado,Max_Diario,Promedio_Ultimos_7_Dias,Dias_Corregidos_Negativos,Nulos_Imputados_Provincia,Duplicados_Eliminados
0,Argentina,10044957,139853,118.86,0,198,0


---------------------------------------------------------------
